## RoundTrip Profile with YAML

### Steps:
 - Get US Core Patient Profile as JSON
 - Convert to YML
 - Convert back to JSON
 - Run in ig-pub to see if works

### Imports

In [ ]:
from json import dumps, loads, load
from pprint import pprint
from requests import get, post, put
import os
from pathlib import Path
from csv import reader as csvreader
from IPython.display import display as Display, HTML, Markdown
from yaml import load as y_load, dump as y_dump
try:
    from yaml import CLoader as Loader, CDumper as Dumper
except ImportError:
    from yaml import Loader, Dumper

### Get US Core Patient Profile as JSON

In [ ]:
my_path = Path.home() / 'Documents' / 'FHIR' / 'US-Core-R4' / 'output'
my_path

In [ ]:
f=sorted(my_path.glob('Struct*patient*.json'))
f

In [ ]:
f[0].as_posix()

In [ ]:
f[0].as_uri()

In [ ]:
r_json=f[0].read_text()
print(r_json)

In [ ]:
r_dict=loads(r_json)
r_dict

### Strip the resouce down to meta + diff (omit text and mapping)

In [ ]:
for i in ['snapshot', 'text',]: # 'mapping']:
    del r_dict[i]
r_dict

### Convert to YML Using the PyYAML Module and Maintain the Order

In [ ]:
r_yml = y_dump(r_dict, Dumper=Dumper, sort_keys=False)
print(r_yml)

### Convert Back to Dict

- change to id and url to 'yaml-rtrip'

In [ ]:
r_dict = y_load(r_yml, Loader=Loader)
r_dict['id'] = 'yaml-rtrip'
r_dict['url'] = 'http://www.fhir.org/guides/test3/StructureDefinition/yaml-rtrip'
r_dict

### ... And to JSON

In [ ]:
r_json = dumps(r_dict, indent=4)

print(r_json)

### Validate in test server

In [ ]:
# *********************** validate Resource ********************************

fhir_test_server = 'http://test.fhir.org/r4'
headers = {
'Accept':'application/fhir+json',
'Content-Type':'application/fhir+json'
}
params = dict(
  # profile = 'http://hl7.org/fhir/us/core/StructureDefinition/us-core-patient' # The official URL for this profile is: http://hl7.org/fhir/us/core/StructureDefinition/us-core-patient
    )
r = post(f'{fhir_test_server}/Questionnaire/$validate', params = params, headers = headers, data = r_json)
display(HTML(f'<h1>Validation output</h1><h3>Status Code = {r.status_code}</h3> {r.json()["text"]["div"]}'))

###  Load into Test IG and Run

In [ ]:
out_path = Path.home() / 'Documents' / 'FHIR' / 'IG-Template2' / 'source' / 'resources' / f'StructureDefinition-{r_dict["id"]}.json'
out_path

In [ ]:
out_path.write_text(r_json)

### Run the ig publisher locally
1. run ig publisher to validate examples **it will take a couple of minutes to run** 

In [ ]:
%%bash
set -e
cd /Users/ehaas/Documents/FHIR/IG-Template2/
path1=/Users/ehaas/Downloads/org.hl7.fhir.igpublisher.jar
path3=/Users/ehaas/Documents/FHIR/IG-tools/
echo "================================================================="
echo === use definition files from relative path ../$SOURCE ===
echo "================================================================="
echo getting rid of .DS_Store files since they gum up the igpublisher....
echo "================================================================="
echo === run definitions maker with optional source directory name as first argument ===
echo === create ig.json and ig.xml in $PWD and ../$SOURCE ===
echo "================================================================="
find . -name '.DS_Store' -type f -delete
python3.5 ${path3}definitions.py
java -jar ${path1} -ig ig.json

### Get qa.html output and display

In [ ]:
p = Path(r'/Users/ehaas/Documents/FHIR/IG-Template2/docs/qa.html')

display(HTML(p.read_text()))